In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import re

**Create Datasets/CSVs of Products Not Found in First Dataset Scraped**

In [3]:
# Ulta_pl = pd.read_csv('/content/drive/MyDrive/Capstone Project - Summer 2024/Cleaned Datasets From Web Scraping - Part 1/Ulta_products_links_only.csv')
# Ulta_pd = pd.read_csv('/content/drive/MyDrive/Capstone Project - Summer 2024/Cleaned Datasets From Web Scraping - Part 2/Ulta_product_description.csv')
# Ulta_i = pd.read_csv('/content/drive/MyDrive/Capstone Project - Summer 2024/Cleaned Datasets From Web Scraping - Part 2/Ulta_ingredients.csv')

# Ulta_pd_extra = Ulta_pl[~Ulta_pl['Product ID'].isin(Ulta_pd['Product ID'])]
# Ulta_i_extra = Ulta_pl[~Ulta_pl['Product ID'].isin(Ulta_i['Product ID'])]

# # Ulta_pd_extra.to_csv('Ulta_product_description_extra.csv', index=False)
# Ulta_i_extra.to_csv('Ulta_ingredients_extra.csv', index=False)

**Check and Combine All Scraped Data**

In [4]:
# Load data into Spark DataFrame
ulta_product_description = pd.read_csv('/content/drive/MyDrive/Capstone Project - Summer 2024/Cleaned Datasets From Web Scraping - Part 2/Ulta_product_description.csv')
ulta_ingredients = pd.read_csv('/content/drive/MyDrive/Capstone Project - Summer 2024/Cleaned Datasets From Web Scraping - Part 2/Ulta_ingredients.csv')
ulta_reviews = pd.read_csv('/content/drive/MyDrive/Capstone Project - Summer 2024/Cleaned Datasets From Web Scraping - Part 2/Ulta_reviews.csv')

ulta_additional_ingredients = pd.read_csv('/content/drive/MyDrive/Capstone Project - Summer 2024/Cleaned Datasets From Web Scraping - Part 2/Ulta_additional_ingredients.csv')
ulta_additional_product_description = pd.read_csv('/content/drive/MyDrive/Capstone Project - Summer 2024/Cleaned Datasets From Web Scraping - Part 2/Ulta_additional_product_description.csv')
ulta_additional_reviews = pd.read_csv('/content/drive/MyDrive/Capstone Project - Summer 2024/Cleaned Datasets From Web Scraping - Part 2/Ulta_additional_reviews.csv')

In [5]:
# Combine DataFrames
ulta_product_description = pd.concat([ulta_product_description, ulta_additional_product_description])
ulta_ingredients = pd.concat([ulta_ingredients, ulta_additional_ingredients])
ulta_reviews = pd.concat([ulta_reviews, ulta_additional_reviews])

print("Product Description DF Count: ", ulta_product_description.shape[0])
print("Ingredients DF Count: ", ulta_ingredients.shape[0])
print("Review DF Count: ", ulta_reviews.shape[0])

Product Description DF Count:  630
Ingredients DF Count:  660
Review DF Count:  435618


In [6]:
# Remove duplicates based on 'Product ID'
ulta_product_description = ulta_product_description.drop_duplicates(subset=['Product ID'])
ulta_ingredients = ulta_ingredients.drop_duplicates(subset=['Product ID'])

print("Product Description DF Count After Deleting Duplicates: ", ulta_product_description.shape[0])
print("Ingredients DF Count After Deleting Duplicates: ", ulta_ingredients.shape[0])

Product Description DF Count After Deleting Duplicates:  630
Ingredients DF Count After Deleting Duplicates:  660


**Remove any Travel Items**

Travel-sized products were removed for Ulta since the full-size and travel-sized bottles share the same reviews.

In [7]:
ulta_products = pd.read_csv('/content/drive/MyDrive/Capstone Project - Summer 2024/Cleaned Datasets From Web Scraping - Part 1/Ulta_products.csv')

# Identify travel-sized products
travel_size_products = ulta_products[ulta_products['Product Name'].str.contains('Travel Size', case=False, na=False)]

# Function to check if a full-sized version exists and get its product ID
def get_full_size_product_id(row, all_products):
    travel_size_name = row['Product Name']
    full_size_name = travel_size_name.replace('Travel Size', '').strip()
    # Escape the full size name to avoid regex interpretation
    full_size_name_escaped = re.escape(full_size_name)
    full_size_product = all_products[all_products['Product Name'].str.contains(full_size_name_escaped, case=False, na=False)]
    if not full_size_product.empty:
        return full_size_product.iloc[0]['Product ID']
    return None

# Create a mapping of travel-sized product IDs to full-sized product IDs
travel_to_full_mapping = {}
for index, row in travel_size_products.iterrows():
    full_size_product_id = get_full_size_product_id(row, ulta_products)
    if full_size_product_id:
        travel_to_full_mapping[row['Product ID']] = full_size_product_id

# New mappings to add or update
new_mappings = {
    'U51': 'U9', 'U77': 'U49', 'U194': 'U205', 'U223': 'U198', 'U226': 'U200',
    'U402': 'U401', 'U393': 'U314', 'U539': 'U535'
}

# Update existing mapping with new mappings
travel_to_full_mapping.update(new_mappings)

# Entries to remove
entries_to_remove = {
    'U160', 'U164', 'U173', 'U192', 'U387', 'U396', 'U397', 'U400', 'U402', 'U580', 'U632', 'U674'
}

# Remove specific entries
for entry in entries_to_remove:
    travel_to_full_mapping.pop(entry, None)  # Use pop to avoid KeyError if the entry doesn't exist

# Replace specific Product IDs in ulta_reviews
ulta_reviews['Product ID'] = ulta_reviews['Product ID'].replace('U463', 'U413')

# Filter out travel-sized products that have a corresponding full-sized version
travel_size_product_ids_with_full_version = list(travel_to_full_mapping.keys())
full_size_product_ids = list(travel_to_full_mapping.values())

# Remove travel-sized products from the DataFrames if their full-sized version exists in those DataFrames
def filter_dataframe(df, travel_to_full_mapping):
    travel_ids = set(travel_to_full_mapping.keys())
    return df[~df['Product ID'].isin(travel_ids)]

ulta_ingredients= filter_dataframe(ulta_ingredients, travel_to_full_mapping)
ulta_reviews = filter_dataframe(ulta_reviews, travel_to_full_mapping)

# Show the results
print("Mapping of travel size product IDs to full-sized product IDs:", travel_to_full_mapping)
print("Updated Product Description DF Count:", ulta_product_description.shape[0])
print("Updated Ingredients DF Count:", ulta_ingredients.shape[0])
print("Updated Review DF Count:", ulta_reviews.shape[0])

Mapping of travel size product IDs to full-sized product IDs: {'U30': 'U6', 'U51': 'U9', 'U77': 'U49', 'U93': 'U5', 'U116': 'U10', 'U118': 'U7', 'U141': 'U8', 'U145': 'U36', 'U149': 'U78', 'U150': 'U27', 'U157': 'U12', 'U161': 'U39', 'U163': 'U16', 'U166': 'U59', 'U169': 'U31', 'U172': 'U40', 'U176': 'U25', 'U177': 'U148', 'U178': 'U135', 'U179': 'U43', 'U180': 'U24', 'U186': 'U182', 'U194': 'U205', 'U220': 'U196', 'U223': 'U198', 'U226': 'U200', 'U230': 'U228', 'U236': 'U231', 'U290': 'U243', 'U317': 'U246', 'U335': 'U242', 'U337': 'U250', 'U342': 'U247', 'U345': 'U245', 'U360': 'U262', 'U363': 'U273', 'U367': 'U282', 'U369': 'U261', 'U373': 'U254', 'U375': 'U269', 'U379': 'U203', 'U383': 'U275', 'U384': 'U256', 'U389': 'U278', 'U390': 'U272', 'U392': 'U264', 'U393': 'U314', 'U394': 'U248', 'U395': 'U340', 'U398': 'U281', 'U448': 'U403', 'U463': 'U413', 'U472': 'U439', 'U482': 'U476', 'U503': 'U494', 'U504': 'U488', 'U505': 'U489', 'U507': 'U480', 'U517': 'U514', 'U539': 'U535', 'U548

In [8]:
ulta_products[ulta_products['Product ID'] == 'U463']['Product Name'].values[0]

'Travel Size Like A Virgin Super Nourishing Coconut & Fig Hair Masque'

In [9]:
ulta_products[ulta_products['Product Name'].str.contains("Like A Virgin Super Nourishing Coconut & Fig Hair Masque", case=False, na=False)]
# {'U51': 'U9', 'U77': 'U49', 'U194': 'U205', 'U223': 'U198', 'U226': 'U200', 'U402': 'U401', 'U393': 'U314', 'U539': 'U535'}
# Remove : {'U160': 'U160', 'U164': 'U164', 'U173': 'U173', 'U192': 'U192', 'U387': 'U387', 'U396': 'U396', 'U397': 'U397', 'U400': 'U400', 'U402': 'U200', 'U580': 'U580', 'U632': 'U632', 'U674': 'U674',}
# Change the product 'U379' 'Product Type' to 'Leave-In Conditioner' in ulta_products

# Keep both for reviews? or just rename in reviews. Keep in set to remove for ingr 'U463': 'U413'

,Brand,Product Name,Price,Rating,Number of Ratings,Link,Product Type,Product ID
412,Coco & Eve,Like A Virgin Super Nourishing Coconut & Fig H...,$39.00,4.7,4432,https://www.ulta.com/p/like-a-virgin-super-nou...,Hair Mask,U413
462,Coco & Eve,Travel Size Like A Virgin Super Nourishing Coc...,$17.00,4.8,1042,https://www.ulta.com/p/travel-size-like-a-virg...,Hair Mask,U463


In [10]:
full_size_product_ids

['U6',
 'U9',
 'U49',
 'U5',
 'U10',
 'U7',
 'U8',
 'U36',
 'U78',
 'U27',
 'U12',
 'U39',
 'U16',
 'U59',
 'U31',
 'U40',
 'U25',
 'U148',
 'U135',
 'U43',
 'U24',
 'U182',
 'U205',
 'U196',
 'U198',
 'U200',
 'U228',
 'U231',
 'U243',
 'U246',
 'U242',
 'U250',
 'U247',
 'U245',
 'U262',
 'U273',
 'U282',
 'U261',
 'U254',
 'U269',
 'U203',
 'U275',
 'U256',
 'U278',
 'U272',
 'U264',
 'U314',
 'U248',
 'U340',
 'U281',
 'U403',
 'U413',
 'U439',
 'U476',
 'U494',
 'U488',
 'U489',
 'U480',
 'U514',
 'U535',
 'U536',
 'U550',
 'U602',
 'U612',
 'U621',
 'U651',
 'U652',
 'U653',
 'U671',
 'U672']

**Remove Products with Less Than 4.5 Rating and 100 Written Reviews.**


In [11]:
# Lists of product IDs to exclude
product_ids_less_100_reviews = ['U112', 'U128', 'U153', 'U274', 'U331', 'U341', 'U347', 'U508', 'U549', 'U560']
product_ids_less_45_rating = ['U26', 'U36', 'U112', 'U128', 'U145', 'U153', 'U274', 'U279', 'U331', 'U341', 'U347', 'U387', 'U477', 'U549', 'U560', 'U603']
product_ids_page_not_found = ['U38', 'U55', 'U62', 'U91', 'U124', 'U149', 'U156', 'U177', 'U184', 'U283', 'U318', 'U336', 'U354', 'U370', 'U385', 'U457', 'U470', 'U472', 'U483', 'U503', 'U590', 'U667']

# Combine all lists
product_ids_to_exclude = set(product_ids_less_100_reviews + product_ids_less_45_rating + product_ids_page_not_found)

# Filter out the rows with the specified product IDs
ulta_product_description = ulta_product_description[~ulta_product_description['Product ID'].isin(product_ids_to_exclude)]
ulta_ingredients = ulta_ingredients[~ulta_ingredients['Product ID'].isin(product_ids_to_exclude)]
ulta_reviews = ulta_reviews[~ulta_reviews['Product ID'].isin(product_ids_to_exclude)]

print("Product Description DF Count: ", ulta_product_description.shape[0])
print("Ingredients DF Count: ", ulta_ingredients.shape[0])
print("Review DF Count: ", ulta_reviews.shape[0])

Product Description DF Count:  616
Ingredients DF Count:  579
Review DF Count:  382431


**Check if there are any products in the reviews and ingredients review that are not in the product description DataFrame.**

System 1 requires `ulta_product_description` and `ulta_reviews` to have the same product IDs. System 2 only requires `ulta_ingredients` and does not require the other two.

In [12]:
# Extract Product IDs
product_desc_ids = set(ulta_product_description['Product ID'].unique())
ingredients_ids = set(ulta_ingredients['Product ID'].unique())
reviews_ids = set(ulta_reviews['Product ID'].unique())

# Find Product IDs in ingredients but not in product descriptions
ingredients_not_in_desc = ingredients_ids - product_desc_ids

# Find Product IDs in product descriptions but not in ingredients
desc_not_in_ingredients = product_desc_ids - ingredients_ids
# Find Product IDs in product descriptions but not in reviews
desc_not_in_rev = product_desc_ids - reviews_ids

# Find Product IDs in reviews but not in ingredients
reviews_not_in_ingredients = reviews_ids - ingredients_ids

# Find Product IDs in reviews but not in product descriptions
reviews_not_in_product_desc = reviews_ids - product_desc_ids

# Convert to lists
desc_not_in_ingredients_list = list(desc_not_in_ingredients)
ingredients_not_in_desc_list = list(ingredients_not_in_desc)
desc_not_in_rev_list = list(desc_not_in_rev)
reviews_not_in_ingredients_list = list(reviews_not_in_ingredients)
reviews_not_in_product_desc_list = list(reviews_not_in_product_desc)

# Show the results
print("Product IDs in product descriptions but not in ingredients:", desc_not_in_ingredients_list)
print("Product IDs in product descriptions but not in reviews:", desc_not_in_rev_list)
print("Product IDs in ingredients but not in product descriptions:", ingredients_not_in_desc_list)
print("Product IDs in reviews but not in ingredients:", reviews_not_in_ingredients_list)
print("Product IDs in reviews but not in product descriptions:", reviews_not_in_product_desc_list)

Product IDs in product descriptions but not in ingredients: ['U178', 'U172', 'U141', 'U394', 'U236', 'U150', 'U290', 'U77', 'U220', 'U379', 'U631', 'U548', 'U383', 'U51', 'U463', 'U166', 'U179', 'U392', 'U176', 'U504', 'U564', 'U180', 'U507', 'U337', 'U163', 'U539', 'U157', 'U116', 'U335', 'U375', 'U226', 'U93', 'U616', 'U118', 'U186', 'U676', 'U384', 'U30', 'U169', 'U395', 'U230', 'U393', 'U363', 'U655', 'U398', 'U482', 'U223', 'U389', 'U681', 'U660', 'U373', 'U342', 'U390', 'U680', 'U367', 'U360', 'U161', 'U619', 'U345', 'U317', 'U657', 'U448', 'U369', 'U194']
Product IDs in product descriptions but not in reviews: ['U172', 'U394', 'U75', 'U206', 'U105', 'U179', 'U176', 'U504', 'U116', 'U375', 'U118', 'U573', 'U416', 'U533', 'U389', 'U681', 'U373', 'U342', 'U619', 'U408', 'U657', 'U448', 'U369', 'U178', 'U150', 'U290', 'U337', 'U163', 'U335', 'U450', 'U93', 'U384', 'U169', 'U592', 'U482', 'U660', 'U390', 'U360', 'U345', 'U317', 'U28', 'U557', 'U631', 'U548', 'U51', 'U166', 'U564', 'U

In [13]:
ingredients_not_in_desc - reviews_not_in_product_desc

{'U225', 'U443'}

In [14]:
reviews_not_in_product_desc - ingredients_not_in_desc

set()

In [15]:
len(ingredients_not_in_desc_list)

27

**Product IDs in product descriptions but not in reviews that were not processed:**


These products consistently had mismatched reviews or incomplete scraping of reviews. However, they will be retained in the product description dataset in case they are needed for system 2.

In [16]:
desc_not_in_rev_list

['U172',
 'U394',
 'U75',
 'U206',
 'U105',
 'U179',
 'U176',
 'U504',
 'U116',
 'U375',
 'U118',
 'U573',
 'U416',
 'U533',
 'U389',
 'U681',
 'U373',
 'U342',
 'U619',
 'U408',
 'U657',
 'U448',
 'U369',
 'U178',
 'U150',
 'U290',
 'U337',
 'U163',
 'U335',
 'U450',
 'U93',
 'U384',
 'U169',
 'U592',
 'U482',
 'U660',
 'U390',
 'U360',
 'U345',
 'U317',
 'U28',
 'U557',
 'U631',
 'U548',
 'U51',
 'U166',
 'U564',
 'U507',
 'U539',
 'U157',
 'U316',
 'U522',
 'U226',
 'U395',
 'U396',
 'U680',
 'U161',
 'U270',
 'U200',
 'U29',
 'U236',
 'U141',
 'U77',
 'U220',
 'U379',
 'U108',
 'U383',
 'U463',
 'U392',
 'U180',
 'U628',
 'U320',
 'U616',
 'U186',
 'U30',
 'U230',
 'U393',
 'U363',
 'U655',
 'U398',
 'U223',
 'U367',
 'U27',
 'U194']

**Product IDs in reviews but not in product descriptions that were not processed:**


1.   Check if items need to be excluded. The output from my code:


```
No skipped product IDs found.
No products with less than 100 reviews found.
No products with less than 4.5 rating found.
```

2.   Run Code to Scrape. Run twice if unsuccessful the first time. For all products, I would get a `Timeout Error`.


3. Manually add in the product information if not too many products.



In [17]:
# ['U365', 'U517', 'U358', 'U617', 'U465', 'U454', 'U138', 'U466', 'U217', 'U683', 'U447', 'U147', 'U311', 'U467', 'U299', 'U98', 'U131', 'U505', 'U673', 'U133', 'U19', 'U462', 'U519', 'U596', 'U635', 'U323', 'U162']
# Create a DataFrame with the new entries
new_entries = pd.DataFrame([
    {
        'Product ID': 'U133',
        'Picture URL': 'https://media.ulta.com/i/ulta/2560998?w=855&h=855&fmt=auto',
        'Description': "Beachwaver's Second Chance Dry Shampoo with rice starch and Brazilian Berry fragrance. Absorbs oil to revive your hair and restore fullness to your roots.",
        'Health Facts': '',
        'Benefits': 'Save time and give your waves an extra day, Absorbs oil with a rice formula for reviving hair and restoring fullness to roots, Brazilian Berry aroma experience',
        'Features': ''
    },
    {
        'Product ID': 'U19',
        'Picture URL': 'https://media.ulta.com/i/ulta/2623814?w=855&h=855&fmt=auto',
        'Description': "For hair that just won't seem to grow. Bondi Boost's HG Shampoo for Thinning Hair is a natural, Australian aloe-based shampoo to gently cleanse without sulphates to nourish your hair and scalp.",
        'Health Facts': 'Clean Ingredients, Cruelty Free, Sustainable Packaging Brand',
        'Benefits': '',
        'Features': ''
    },
    {
        'Product ID': 'U365',
        'Picture URL': 'https://media.ulta.com/i/ulta/2563398?w=855&h=855&fmt=auto',
        'Description': "SheaMoisture's Manuka Honey & Yogurt Hydrate + Repair Conditioner with mafura and baobab oils softens, detangles and instantly infuses abused hair with intensive moisture.",
        'Health Facts': '',
        'Benefits': '',
        'Features': ''
    },
    {
        'Product ID': 'U465',
        'Picture URL': 'https://media.ulta.com/i/ulta/2541274?w=855&h=855&fmt=auto',
        'Description': "Wella INVIGO Vibrant Color Mask for Coarse Hair is an intensive hair mask that enhances and protects color.",
        'Health Facts': '',
        'Benefits': 'Prevents shade fade, Defends against damaging environmental aggressors, Helps to maintain vibrancy and shine for up to 7 weeks',
        'Features': ''
    },
    {
        'Product ID': 'U673',
        'Picture URL': 'https://media.ulta.com/i/ulta/2521655?w=855&h=855&fmt=auto',
        'Description': "SheaMoisture¿s Manuka Honey & Yogurt Hydrate + Repair Protein-Strong Treatment with Mafura & Baobab Oils is powered by reparative proteins and butters, in a nutrient-rich strengthening cream to naturally reinforce and revitalize over-processed, abused hair fibers.",
        'Health Facts': '',
        'Benefits': '',
        'Features': ''
    },
    {
        'Product ID': 'U447',
        'Picture URL': 'https://media.ulta.com/i/ulta/2290595?w=855&h=855&fmt=auto',
        'Description': "SheaMoisture's Manuka Honey & Mafura Oil Intensive Hydration Hair Masque intensely conditions. Restores and locks in moisture while deeply conditioning. Smoothes and fortifies follicles for stronger, healthier frizz-free hair.",
        'Health Facts': '',
        'Benefits': '',
        'Features': ''
    },
    {
        'Product ID': 'U138',
        'Picture URL': 'https://media.ulta.com/i/ulta/2541299?w=855&h=855&fmt=auto',
        'Description': "Wella INVIGO Blonde Recharge Color Refreshing Shampoo, for cool blonde tones, helps to fight brass and maintain the color of blonde tones.",
        'Health Facts': '',
        'Benefits': 'For cool blonde tones refreshes and maintains the brightness and vibrancy of natural or highlighted blondes, This purple shampoo prevents natural or colored blonde hair from yellowing and helps eliminate brassiness, leaving it soft to the touch, Apply to wet hair and lather, Leave for 3-5 minutes, then rinse thoroughly, Use with our Cool Blonde Shampoo for maximum effect',
        'Features': ''
    },
    {
        'Product ID': 'U217',
        'Picture URL': 'https://media.ulta.com/i/ulta/2281363?w=855&h=855&fmt=auto',
        'Description': "SheaMoisture's Jamaican Black Castor Oil Strengthen & Restore Leave-In Conditioner with Shea Butter, Peppermint & Apple Cider Vinegar intensely smooths & nourishes natural, chemically processed or heat styled hair.",
        'Health Facts': '',
        'Benefits': '',
        'Features': ''
    },
    {
        'Product ID': 'U299',
        'Picture URL': 'https://media.ulta.com/i/ulta/2257337?w=855&h=855&fmt=auto',
        'Description': "It's a 10 Miracle Silk Express Miracle Silk Conditioner cuts down drying time, ultra-silky formula, strengthens hair, improves elasticity, adds vibrancy, enhances body, environmental protector, ultra shine & manageability.",
        'Health Facts': 'Cruelty Free',
        'Benefits': 'Cuts down drying time, Ultra-silky formula, Strengthens hair, Improves elasticity, Adds vibrancy, Enhances body in hair, Environmental protector, Ultra shine & manageability, Deeply nourishing formula, Prolongs color',
        'Features': "Few compounds deliver the luxe conditioning and weightless body of silk - which is why It's a 10's Silk Express Miracle Silk Conditioner uses lightweight natural silk as its main ingredient, No other single natural ingredient delivers the same performance as It's a 10's silk proteins for reducing drying time, eliminating frizz, locking in smoothness, and enhancing hair's overall strength, When used regularly, this silk-infused daily conditioner leaves hair ultra silky, smooth, and shiny"
    },
    {
        'Product ID': 'U162',
        'Picture URL': 'https://media.ulta.com/i/ulta/2563350?w=855&h=855&fmt=auto',
        'Description': "Reduce breakage caused by heat styling with SheaMoisture's Manuka Honey & Yogurt Hydrate + Repair Shampoo with mafura and baobab oils. This sulfate-free shampoo works to gently cleanse, hydrate, and repair abused hair.",
        'Health Facts': '',
        'Benefits': '',
        'Features': ''
    },
    {
        'Product ID': 'U467',
        'Picture URL': 'https://media.ulta.com/i/ulta/2600519?w=855&h=855&fmt=auto',
        'Description': "Wella's ColorMotion+ Structure+ Mask is an intensive restructuring mask for colored hair that provides strengthened hair structure, shine and lasting manageability. With bond structuring agent from WellaPlex.",
        'Health Facts': '',
        'Benefits': 'Helps to reconstruct inner hair bonds, Helps to make the hair stronger',
        'Features': 'With WellaPlex bonding agent.'
    },
    {
        'Product ID': 'U311',
        'Picture URL': 'https://media.ulta.com/i/ulta/2541279?w=855&h=855&fmt=auto',
        'Description': "Wella INVIGO Vibrant Color Conditioner For Coarse Hair provides instant color vibrancy, for up to 7 weeks color protection with regular use of Brilliance Shampoo.",
        'Health Facts': '',
        'Benefits': 'Prevents shade fade, Smoothes the hair cuticle for a high-shine finish, Helps to maintain vibrancy and shine for up to 7 weeks, For coarse, colored hair',
        'Features': ''
    },
    {
        'Product ID': 'U131',
        'Picture URL': 'https://media.ulta.com/i/ulta/2541301?w=855&h=855&fmt=auto',
        'Description': "Wella INVIGO Volume Boost Bodifying Shampoo provides lightweight body and removes residues from hair.",
        'Health Facts': '',
        'Benefits': 'Lightweight formula optimal for fuller-looking hair, With cotton extract for softness and stability, Spring force polymers to help bodify the hair, For normal to fine hair lacking volume',
        'Features': ''
    },
    {
        'Product ID': 'U462',
        'Picture URL': 'https://media.ulta.com/i/ulta/2582271?w=855&h=855&fmt=auto',
        'Description': "The Wella Professionals NUTRICURLS Deep Treatment is an intense nourishing treatment that helps to prevent frizz for luscious, touchable curls and waves.",
        'Health Facts': '',
        'Benefits': 'Conditioning treatment for intense nourishment, Helps prevent frizz for luscious, touchable curls and waves, High level of nourishing and softening ingredients',
        'Features': ''
    },
    {
        'Product ID': 'U519',
        'Picture URL': 'https://media.ulta.com/i/ulta/2215566?w=855&h=855&fmt=auto',
        'Description': "Inspired by natures water-resistant camellia, Biolage Smooth Proof Leave-in Cream for frizzy hair provides humidity control and anti-frizz smoothness. Packaging may vary.",
        'Health Facts': 'Vegan, Cruelty Free, Clean Ingredients, Sustainable Packaging Brand',
        'Benefits': '',
        'Features': ''
    },
    {
        'Product ID': 'U635',
        'Picture URL': 'https://media.ulta.com/i/ulta/2550677?w=855&h=855&fmt=auto',
        'Description': "Wella Professionals NUTRICURLS EIMI Soft Twirl Anti-Frizz Foam makes your waves touchable with volume, definition and nourishment.",
        'Health Facts': '',
        'Benefits': 'Anti-Frizz Spray, Seals in nourishment, locks out humidity, and reduces frizz for bouncy, defined, manageable curls or waves, Helps to give curls and waves definition, better manageability, luscious softness, and bounciness, 72 hour anti-frizz for Curls & Waves, Spray on damp or dry hair, Can be reapplied anytime during the day to reactivate curls',
        'Features': ''
    },
    {
        'Product ID': 'U596',
        'Picture URL': 'https://media.ulta.com/i/ulta/2611929?w=855&h=855&fmt=auto',
        'Description': "Sebastian Penetraitt Overnight Repair Serum with Hyaluronic Acid and Niacinamide that penetrates and helps to repair your hair while you sleep.",
        'Health Facts': '',
        'Benefits': 'Wake up to your hair looking & feeling replenished and ready for limitless styling, Penetrates overnight, helping to repair the hair cuticle from the inside out, Provides instant moisturization to plump dry, damaged hair, Can be used on dry or damp hair',
        'Features': ''
    },
    {
        'Product ID': 'U683',
        'Picture URL': 'https://media.ulta.com/i/ulta/2113777?w=855&h=855&fmt=auto',
        'Description': "CHI Infra Therma Protective Treatment moisturizes and reconstructs the inner structure of the hair.",
        'Health Facts': 'Cruelty Free',
        'Benefits': 'Creates healthier, more manageable hair by strengthening the inner structure of the hair, improving strength and elasticity, Suitable for all hair types, Helps protect color, Adds superior shine and softness, Contains Ionic and Catatonic Hydration Interlink technology',
        'Features': ''
    },
    {
        'Product ID': 'U617',
        'Picture URL': 'https://media.ulta.com/i/ulta/2579275?w=855&h=855&fmt=auto',
        'Description': "Ouidad's Coil Infusion Defining Gel creates lasting curl definition and touchable hold while adding nourishment.",
        'Health Facts': 'Cruelty Free, Sustainable Packaging Brand',
        'Benefits': '',
        'Features': ''
    },
    {
        'Product ID': 'U466',
        'Picture URL': 'https://media.ulta.com/i/ulta/2543648?w=855&h=855&fmt=auto',
        'Description': "The SheaMoisture Coconut & Hibiscus Hair Masque is a deep treatment masque that provides curl enhancing hydration and instant, brilliant shine while defending against frizz for defined and enhanced natural curls.",
        'Health Facts': '',
        'Benefits': '',
        'Features': ''
    },
    {
        'Product ID': 'U98',
        'Picture URL': 'https://media.ulta.com/i/ulta/2562979?w=855&h=855&fmt=auto',
        'Description': "The dpHUE Apple Cider Vinegar Soothing Shampoo gently cleanses hair, rebalances scalp pH and keeps hair color vibrant. This SLS and SLES sulfate-free shampoo is infused with amino acids, panthenol, lavender, ginseng and ginger extracts. For all hair colors and types.",
        'Health Facts': 'Vegan, Cruelty Free, Clean Ingredients',
        'Benefits': 'Helps cleanse and remove impurities from hair, Helps soothe and moisturize dry, itchy and flaky scalps, Does not strip natural oils essential to scalp and hair health, Helps rebalance pH to reduce frizz & increase shine, Color safe formula, Cruelty-free and vegan',
        'Features': ''
    },
    {
        'Product ID': 'U454',
        'Picture URL': 'https://media.ulta.com/i/ulta/2541269?w=855&h=855&fmt=auto',
        'Description': "Wella INVIGO Vibrant Color Mask for Normal Hair is an intensive hair mask that enhances and protects color.",
        'Health Facts': '',
        'Benefits': 'Prevents shade fade, Defends against damaging environmental aggressors, Helps to maintain vibrancy and shine for up to 7 weeks',
        'Features': ''
    },
    {
        'Product ID': 'U358',
        'Picture URL': 'https://media.ulta.com/i/ulta/2298787?w=855&h=855&fmt=auto',
        'Description': "SheaMoisture Jamaican Black Castor Oil Strengthen & Restore Conditioner for damaged hair detangles and restores moisture without weighing hair down. Perfect for those who regularly color, straighten or perm their hair as well as kinky, curly and wavy natural styles.",
        'Health Facts': '',
        'Benefits': '',
        'Features': ''
    },
    {
        'Product ID': 'U323',
        'Picture URL': 'https://media.ulta.com/i/ulta/2550673?w=855&h=855&fmt=auto',
        'Description': "The Wella Professionals NUTRICURLS Waves & Curls Detangling Conditioner is for lightweight definition.",
        'Health Facts': '',
        'Benefits': 'Detangling conditioner for curly and wavy hair, For lightweight definition, Apply to clean, damp hair and distribute evenly, Rinse thoroughly',
        'Features': ''
    },
    {
        'Product ID': 'U147',
        'Picture URL': 'https://media.ulta.com/i/ulta/2560287?w=855&h=855&fmt=auto',
        'Description': "The Wella Professionals NUTRICURLS Curls Micellar Shampoo is a shampoo for curls that gently removes impurities while providing definition, nourishment and anti-frizz.",
        'Health Facts': '',
        'Benefits': 'Helps to give curls and waves definition, Anti-frizz effect, Better manageability, luscious softness, and bounciness',
        'Features': ''
    },
    {
        'Product ID': 'U443',
        'Picture URL': 'https://media.ulta.com/i/ulta/2569131?w=855&h=855&fmt=auto',
        'Description': "Treat your tresses to a creamy, whipped deep conditioning with L'ange Rehab Hydrating Masque, a middleweight mask fortified with keratin and a blend of botanical oils.",
        'Health Facts': '',
        'Benefits': '',
        'Features': ''
    },
    {
        'Product ID': 'U225',
        'Picture URL': 'https://media.ulta.com/i/ulta/2595082?w=855&h=855&fmt=auto',
        'Description': "The Mielle Organics White Peony Leave-in Conditioner is an ultra moisturizing lightweight leave-in conditioner, packed with botanical extracts and certified organic ingredients to give your hair the moisture and protection it needs.",
        'Health Facts': 'Clean Ingredients, Cruelty Free, Give Back',
        'Benefits': '',
        'Features': ''
    },
])

# Concatenate the new entries with the original DataFrame
ulta_product_description = pd.concat([ulta_product_description, new_entries], ignore_index=True)
print("Number of Products in Product Description DF:", len(ulta_product_description['Product ID'].unique()))
print("Number of Products in Review DF:", len(ulta_reviews['Product ID'].unique()))

Number of Products in Product Description DF: 643
Number of Products in Review DF: 557


**Combined the 'Benefits' and 'Features' Column to create a 'Highlights' Column.**

In [18]:
# Combine the 'Benefits' and 'Features' columns into a new 'Highlights' column
ulta_product_description['Highlights'] = ulta_product_description['Benefits'].fillna('') + ' ' + ulta_product_description['Features'].fillna('')

# Drop the original 'Benefits' and 'Features' columns
ulta_product_description.drop(['Benefits', 'Features'], axis=1, inplace=True)

# Show the updated DataFrame
ulta_product_description.head()

,Product ID,Picture URL,Description,Health Facts,Highlights
0,U1,https://media.ulta.com/i/ulta/2564029?w=78&h=7...,The NatureLab. Tokyo Perfect Clean 2-In-1 Scal...,"Cruelty Free, Sustainable Packaging Brand, Giv...","A refreshingly light, soft scent with notes o..."
1,U2,https://media.ulta.com/i/ulta/2607939?w=78&h=7...,Color Wow Dream Filter for Picture-Perfect Col...,NaN,"Fastest, easiest 3-minute hair detox, Removes ..."
2,U3,https://media.ulta.com/i/ulta/2623373?w=78&h=7...,Viviscal Volumizing Dry Shampoo refreshes hair...,Sustainable Packaging Brand,Instantly refreshes and revives hair by removi...
3,U4,https://media.ulta.com/i/ulta/2620116?w=78&h=7...,Garnier Fructis Hair Filler Moisture Repair Sh...,"Vegan, Cruelty Free",Garnier Frutis sulfate-free Hair Filler Hyalur...
4,U5,https://media.ulta.com/i/ulta/2273894?w=78&h=7...,Biolage Color Last Shampoo is essential for ma...,"Vegan, Cruelty Free, Sustainable Packaging Brand",Low pH shampoo for color treated hair helps pr...


**Product IDs in product descriptions but not in ingredients that were not processed:**

'U676' is not retrieved since the website did not list the ingredients.

**Check if Product IDs in ulta_ingredients and ulta_reviews are in ulta_product_description.**


In [19]:
# Extract unique product IDs
product_desc_ids = set(ulta_product_description['Product ID'].unique())
ingredients_ids = set(ulta_ingredients['Product ID'].unique())
reviews_ids = set(ulta_reviews['Product ID'].unique())

# Find product IDs in ulta_ingredients that are not in ulta_product_description
ingredients_not_in_desc = ingredients_ids - product_desc_ids

# Find product IDs in ulta_reviews that are not in ulta_product_description
reviews_not_in_desc = reviews_ids - product_desc_ids

# Print the results
print("Product IDs in ingredients but not in product descriptions:", list(ingredients_not_in_desc))
print("Product IDs in reviews but not in product descriptions:", list(reviews_not_in_desc))

# Additional check for a more comprehensive understanding
if not ingredients_not_in_desc:
    print("All unique product IDs in ulta_ingredients are present in ulta_product_description.")
else:
    print(f"{len(ingredients_not_in_desc)} unique product IDs in ulta_ingredients are missing in ulta_product_description.")

if not reviews_not_in_desc:
    print("All unique product IDs in ulta_reviews are present in ulta_product_description.")
else:
    print(f"{len(reviews_not_in_desc)} unique product IDs in ulta_reviews are missing in ulta_product_description.")

Product IDs in ingredients but not in product descriptions: []
Product IDs in reviews but not in product descriptions: []
All unique product IDs in ulta_ingredients are present in ulta_product_description.
All unique product IDs in ulta_reviews are present in ulta_product_description.


**Download Updated Ulta CSVs (4)**

In [20]:
# ulta_products.to_csv('ulta_products_final.csv', index=False)
# ulta_product_description.to_csv('ulta_product_description_final.csv', index=False)
# ulta_ingredients.to_csv('ulta_ingredients_final.csv', index=False)
# ulta_reviews.to_csv('ulta_reviews_final.csv', index=False)

# **Target Preprocessing**



In [21]:
# Load data into Spark DataFrame
Target_products = pd.read_csv('/content/drive/MyDrive/Capstone Project - Summer 2024/Cleaned Datasets From Web Scraping - Part 1/Target_products.csv')
Target_product_description = pd.read_csv('/content/drive/MyDrive/Capstone Project - Summer 2024/Cleaned Datasets From Web Scraping - Part 2/Target_product_description.csv')
Target_ingredients = pd.read_csv('/content/drive/MyDrive/Capstone Project - Summer 2024/Cleaned Datasets From Web Scraping - Part 2/Target_ingredients.csv')

print("Products DF Count: ", Target_products.shape[0])
print("Product Description DF Count: ", Target_product_description.shape[0])
print("Ingredients DF Count: ", Target_ingredients.shape[0])

Products DF Count:  543
Product Description DF Count:  469
Ingredients DF Count:  439


**Identify Products with 'Ulta Beauty' in the Ingredient and Product Description DataFrames**

Any products with 'Ulta Beauty' that are not in the `ulta_ingredients` DataFrame is kept. The rest is removed from the Target ingredients and product description DataFrames.

In [22]:
# Identify products with 'Ulta Beauty' in their name
ulta_beauty_Target_products = Target_products[Target_products['Product Name'].str.contains('Ulta Beauty', case=False, na=False)]

# Create a list of product IDs for these products
ulta_beauty_Target_product_ids = ulta_beauty_Target_products['Product ID'].tolist()

# Define the product IDs to keep
products_to_keep = ['T21', 'T74', 'T93', 'T95', 'T103', 'T109', 'T112', 'T167', 'T176', 'T187',
                    'T294', 'T324', 'T382', 'T400', 'T416', 'T427', 'T433', 'T503', 'T511', 'T515', 'T516', 'T542']

# Adjust the list to remove by excluding the IDs you want to keep
products_to_remove = [pid for pid in ulta_beauty_Target_product_ids if pid not in products_to_keep]

# Remove these products from the original DataFrames
Target_product_description = Target_product_description[~Target_product_description['Product ID'].isin(products_to_remove)]
Target_ingredients = Target_ingredients[~Target_ingredients['Product ID'].isin(products_to_remove)]

# Show the results
print("List of product IDs with 'Ulta Beauty' in their name:", ulta_beauty_Target_product_ids)
print("List of product IDs removed:", products_to_remove)
print("List of product IDs kept:", products_to_keep)
print("Updated Target Product Description DF Count:", Target_product_description.shape[0])
print("Updated Target Ingredients DF Count:", Target_ingredients.shape[0])

List of product IDs with 'Ulta Beauty' in their name: ['T5', 'T7', 'T19', 'T21', 'T23', 'T74', 'T93', 'T95', 'T98', 'T99', 'T100', 'T102', 'T103', 'T109', 'T112', 'T113', 'T114', 'T162', 'T167', 'T176', 'T182', 'T187', 'T208', 'T209', 'T210', 'T212', 'T213', 'T214', 'T215', 'T294', 'T313', 'T322', 'T324', 'T325', 'T326', 'T353', 'T365', 'T367', 'T371', 'T382', 'T386', 'T400', 'T401', 'T406', 'T411', 'T414', 'T415', 'T416', 'T427', 'T428', 'T430', 'T433', 'T437', 'T448', 'T449', 'T455', 'T457', 'T466', 'T468', 'T472', 'T476', 'T480', 'T491', 'T496', 'T503', 'T511', 'T515', 'T516', 'T524', 'T537', 'T542']
List of product IDs removed: ['T5', 'T7', 'T19', 'T23', 'T98', 'T99', 'T100', 'T102', 'T113', 'T114', 'T162', 'T182', 'T208', 'T209', 'T210', 'T212', 'T213', 'T214', 'T215', 'T313', 'T322', 'T325', 'T326', 'T353', 'T365', 'T367', 'T371', 'T386', 'T401', 'T406', 'T411', 'T414', 'T415', 'T428', 'T430', 'T437', 'T448', 'T449', 'T455', 'T457', 'T466', 'T468', 'T472', 'T476', 'T480', 'T491',

Below Code is Used to Find `products_to_keep`

In [23]:
# Check Target Product Name
Target_products[Target_products['Product ID'] == 'T542']['Product Name'].values[0]

'VERB Curl Foaming Gel - 6.7 fl oz - Ulta Beauty'

In [24]:
Target_products[Target_products['Product ID'] == 'T472']['Link'].values[0]

'https://www.target.com/p/bumble-and-bumble-hairdresser-s-invisible-oil-primer-ulta-beauty/-/A-84175044?preselect=82572575#lnk=sametab'

In [25]:
ulta_products[ulta_products['Product ID'] == 'U536']['Link'].values[0]

'https://www.ulta.com/p/hairdressers-invisible-oil-frizz-reducing-hair-oil-xlsImpprod16451024?sku=2516143'

In [26]:
# ['U30', 'U51', 'U77', 'U93', 'U116', 'U118', 'U141', 'U145', 'U149', 'U150', 'U157', 'U160', 'U161', 'U163', 'U164', 'U166', 'U169', 'U172', 'U173', 'U176', 'U177', 'U178', 'U179', 'U180', 'U186', 'U192', 'U194', 'U220', 'U223', 'U226', 'U229', 'U230', 'U236', 'U290', 'U317', 'U335', 'U337', 'U342', 'U345', 'U360', 'U363', 'U367', 'U369', 'U373', 'U375', 'U379', 'U383', 'U384', 'U389', 'U390', 'U392', 'U393', 'U394', 'U395', 'U396', 'U397', 'U398', 'U400', 'U402', 'U448', 'U463', 'U472', 'U482', 'U503', 'U504', 'U505', 'U507', 'U517', 'U539', 'U548', 'U564', 'U580', 'U616', 'U619', 'U631', 'U632', 'U655', 'U657', 'U660', 'U674', 'U680', 'U681']
ulta_ingr_product_names = ulta_products[ulta_products['Product ID'].isin(ulta_ingredients['Product ID'])]
ulta_ingr_product_names[ulta_ingr_product_names['Product Name'].str.contains('Edge Control Hair Wax', case=False, na=False)]
# These are not in ulta_ingredients but everything else is. Keep these & exclude the rest:
#  'T21', 'T74', 'T93', T95', 'T103', 'T109', 'T112', 'T167',  'T176', 'T187', 'T294', 'T324', 'T382', 'T400', 'T416', 'T427', 'T433', 'T503', 'T511', 'T515', 'T516', 'T542'

,Brand,Product Name,Price,Rating,Number of Ratings,Link,Product Type,Product ID


In [27]:
ulta_ingr_product_names[ulta_ingr_product_names['Brand'].str.contains('OUAI', case=False, na=False)]

,Brand,Product Name,Price,Rating,Number of Ratings,Link,Product Type,Product ID
405,OUAI,Fine To Medium Hair Treatment Masque,$38.00,4.6,540,https://www.ulta.com/p/fine-medium-hair-treatm...,Hair Mask,U406
417,OUAI,Thick Hair Treatment Masque,$38.00,4.5,445,https://www.ulta.com/p/thick-hair-treatment-ma...,Hair Mask,U418
475,OUAI,Anti-Frizz Crème,$28.00,4.7,1803,https://www.ulta.com/p/anti-frizz-creme-pimpro...,Cream,U476
534,OUAI,Hair Oil,$30.00,4.6,1812,https://www.ulta.com/p/hair-oil-pimprod2012515...,Oil,U535


**Remove Travel Size Products**

Unlike Ulta, reviews for full-size and travel-size versions of the same product differ at Target. Therefore, the initial plan was to utilize reviews from both sizes to enrich the review data. However, since I was unable to scrape Target reviews, I will now focus solely on the product's ingredients. For this purpose, I will use the full-size product unless only the travel-size version has been scraped.

In [28]:
# Identify travel-sized products
Target_ingredients_products = Target_products[Target_products['Product ID'].isin(Target_ingredients['Product ID'])]
travel_size_products = Target_ingredients_products[Target_ingredients_products['Product Name'].str.contains('Travel Size', case=False, na=False)]
travel_size_products

,Product Name,Brand,Price,Rating,Number of Ratings,Link,Product Type,Product ID
296,Tresemme Moisture Rich Conditioner -Travel Siz...,Tresemme,$1.99,4.5,920,https://www.target.com/p/tresemme-moisture-ric...,Rinse-Out Conditioner,T297


In [29]:
Target_ingredients_products[Target_ingredients_products['Brand'].str.contains('Tresemme', case=False, na=False)]

,Product Name,Brand,Price,Rating,Number of Ratings,Link,Product Type,Product ID
12,Tresemme Flawless Curls Moisturizing Shampoo F...,Tresemme,$5.99,4.5,891,https://www.target.com/p/tresemme-flawless-cur...,Shampoo,T13
124,Tresemme Cruelty-free Keratin Repair Shampoo f...,Tresemme,$5.99,4.6,151,https://www.target.com/p/tresemme-cruelty-free...,Shampoo,T125
193,Tresemme Flawless Curls Moisturizing Condition...,Tresemme,$5.99,4.6,1019,https://www.target.com/p/tresemme-flawless-cur...,Rinse-Out Conditioner,T194
227,Tresemme Moisture Rich with Vitamin E Conditioner,Tresemme,$5.99 - $7.99,4.5,1410,https://www.target.com/p/tresemme-moisture-ric...,Rinse-Out Conditioner,T228
282,Tresemme Silky & Smooth Anti-Frizz Conditioner...,Tresemme,$7.99,4.6,324,https://www.target.com/p/tresemme-silky-38-smo...,Rinse-Out Conditioner,T283
296,Tresemme Moisture Rich Conditioner -Travel Siz...,Tresemme,$1.99,4.5,920,https://www.target.com/p/tresemme-moisture-ric...,Rinse-Out Conditioner,T297
310,Tresemme Cruelty-free Keratin Repair Condition...,Tresemme,$5.99,4.6,156,https://www.target.com/p/tresemme-cruelty-free...,Rinse-Out Conditioner,T311
315,Tresemme Cruelty-free Keratin Smooth Color Con...,Tresemme,$5.99,4.5,158,https://www.target.com/p/tresemme-cruelty-free...,Rinse-Out Conditioner,T316
316,Tresemme Color Revitalize Conditioner for Colo...,Tresemme,$5.99,4.6,304,https://www.target.com/p/tresemme-color-revita...,Rinse-Out Conditioner,T317
409,Tresemme Flawless Curls Combing Hair Cream - 1...,Tresemme,$6.99,4.5,508,https://www.target.com/p/tresemme-flawless-cur...,Cream,T410


In [30]:
Target_products[Target_products['Product ID'] == 'T297']['Link'].values[0]

'https://www.target.com/p/tresemme-moisture-rich-conditioner-travel-size-3-fl-oz/-/A-15143490#lnk=sametab'

This one travel-size product can be removed since its full-size version is within the `Target_ingredients`. This would have been kept within the target reviews dataframe if it had been completed.

In [31]:
Target_product_description = Target_product_description[Target_product_description['Product ID'] != 'T297']
Target_ingredients = Target_ingredients[Target_ingredients['Product ID'] != 'T297']

**Check which Product ID in the review dataframe is not in the product description dataframe.**

These two dataframes should have the same Product IDs since they were scraped simultaneously.

In [32]:
# Extract Product IDs
T_product_desc_ids = set(Target_product_description['Product ID'].unique())
T_ingredients_ids = set(Target_ingredients['Product ID'].unique())

# Find Product IDs in ingredients but not in product descriptions
T_ingredients_not_in_desc = T_ingredients_ids - T_product_desc_ids

# Find Product IDs in product descriptions but not in ingredients
T_desc_not_in_ingredients = T_product_desc_ids - T_ingredients_ids

# Convert to lists
T_ingredients_not_in_desc_list = list(T_ingredients_not_in_desc)
T_desc_not_in_ingredients_list = list(T_desc_not_in_ingredients)

# Show the results
print("Product IDs in ingredients but not in product descriptions:", T_ingredients_not_in_desc_list)
print("Product IDs in product descriptions but not in ingredients:", T_desc_not_in_ingredients_list)

Product IDs in ingredients but not in product descriptions: []
Product IDs in product descriptions but not in ingredients: ['T171', 'T432', 'T88', 'T427', 'T142', 'T341', 'T154', 'T115', 'T403', 'T513', 'T464', 'T489', 'T431', 'T349', 'T423', 'T542', 'T380', 'T26', 'T450', 'T404', 'T398', 'T516', 'T6', 'T426', 'T417', 'T379']


The Product IDs found in the product description DataFrame but not in the ingredients DataFrame were not scraped because their product pages lacked ingredient information.

**Download Updated Target CSVs (2)**

In [33]:
# Target_product_description.to_csv('Target_product_description_final.csv', index=False)
# Target_ingredients.to_csv('Target_ingredients_final.csv', index=False)